In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import os
import matplotlib.pyplot as plt

import scipy.signal as signal
import imageio as io
from tqdm import tqdm
import cv2
import shutil

import sys
sys.path.insert(1, '../numphly')
import numphly as nly

In [2]:
dirloc = '../../../../../../Documents/'
fileloc_set   = np.load('experiments_directory.npy')
num_file = len(fileloc_set);
videofile_set = np.load('experiments_well_setup.npy')
experiments_index = np.load('experiments_index.npy')
print(fileloc_set, len(fileloc_set))
print("")
print(videofile_set)
print("")
print(experiments_index)
sex_name      = ['F','M']
genotype_name = ['RxT', 'RxW', 'TxW']
frame_rate = 60

experiments_mscore = np.load('experiments_mscore.npy',allow_pickle='TRUE').item()
experiments_mscore_1 = np.load('experiments_mscore.npy',allow_pickle='TRUE').item()
cropped_recording_folder= 'cropped_recording_for_tracking'
output_fileloc = '../../FlyBehaviorSet/'
OG_recording_folder= 'original_recording_for_tracking'
converted_folder = 'Converted/R72_videos/'

['5_8_24/5_8_24_R72A10TrpA1Expt1_video_20240508_130433_636/'
 '5_8_24/5_8_24_R72A10TrpA1Expt2_video_20240508_142937_318/'
 '5_9_24/5_9_24_r72A10_expt1_video_20240509_095759_237/'
 '5_9_24/5_9_24_r72a10_expt2_5wellexpt_video_20240509_115237_346/'
 '5_13_24/5_13_24_r72A10_expt1_video_20240513_102826_071/'
 '5_13_24/5_13_24_r72A10_expt2_video_20240513_121809_465/'
 '5_13_24/5_13_24_r72A10_expt3_video_20240513_141847_637/'
 '5_15_24/5_15_24_r72a10_expt1_video_20240515_144359_162/'
 '5_17_24/5_17_24_r72a10_video_20240517_125705_729/'
 '7_23_24/7_23_24_expt1_video_20240723_094225_800/'
 '7_23_24/7_23_24_expt2_video_20240723_130206_295/'
 '7_24_24/7_24_24_extp1_video_20240724_102123_758/'
 '7_24_24/7_24_24_extp2_video_20240724_120325_610/'
 '7_24_24/7_24_24_extp3_video_20240724_140346_190/'
 '7_24_24/7_24_24_extp4_video_20240724_144349_189/'] 15

['A1' 'B1' 'C1' 'D1' 'A2' 'B2' 'C2' 'D2']

['05_08_24_exp1' '05_08_24_exp2' '05_09_24_exp1' '05_09_24_exp2'
 '05_13_24_exp1' '05_13_24_exp2' '05_13_

In [3]:

first_frame = 20000
last_frame = 125000
common_difference = frame_rate*30
event = np.arange(first_frame,last_frame, common_difference)
np.set_printoptions(precision=1)
{}
ii = 0
for i in range(num_file):
# for i in range(len(experiments_mscore)):
# for i in range(1,num_file):
    experiment = experiments_mscore[fileloc_set[i]]
    sexes = experiment['sex']
    gtypes = experiment['genotype']
    event_set_circle = []
    if "events" in experiment:
        events = experiment['events']
        experiment_index = experiments_index[i]
        print(experiment_index)
        for j in range(8):
#         for j in range(1):
            well_arena = np.empty((0,3))
            well_food  = np.empty((0,3))
            well = videofile_set[j]
            sex = sexes[j]
            gtype = gtypes[j]
            output_file_prefix = 'v_'+experiment_index+'_'+well
            if sex != -1:
                # clipping videos for labeling and move them into a single directory
                video   = output_fileloc + OG_recording_folder +'/R72/'+output_file_prefix+'.mp4'
                
#                 video   = output_fileloc+converted_folder+fileloc_set[i]+'well_'+videofile_set[j]+'_ts.mp4'

                cap = cv2.VideoCapture(video)
                length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
                n_circles = 0
                for region in event:
                    cap.set(cv2.CAP_PROP_POS_FRAMES, region)
                    res, frame = cap.read()
                    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                    gray_blurred = cv2.medianBlur(gray, 5)
                    arena = cv2.HoughCircles(gray_blurred, cv2.HOUGH_GRADIENT, 1, 20, 
                                               param1=50, param2=50, minRadius=225, maxRadius=245)
#                         food = cv2.HoughCircles(gray_blurred, cv2.HOUGH_GRADIENT, 1, 20, 
#                                                    param1=55, param2=55, minRadius=50, maxRadius=100)
                    if arena is not None:
                        well_arena = np.vstack([well_arena, (arena[0,:])[0,:]])
                        n_circles = n_circles+1
                    if n_circles >= 50:
                        break
                cap.release()
                
                cv2.destroyAllWindows()
                circle = np.mean(well_arena, axis = 0)
                circle = np.round(circle).astype("int")
                print(output_file_prefix, circle, well_arena.shape)
                cv2.circle(frame, (circle[0], circle[1]), 240, (0, 255, 0), 2)
                # Draw the center of the circle
                cv2.circle(frame, (circle[0], circle[1]), 2, (0, 0, 255), 3)
                image   = output_fileloc + cropped_recording_folder +'/'+output_file_prefix+'_radius.png'
                cv2.imwrite(image, frame, [cv2.IMWRITE_PNG_COMPRESSION, 0])
                ii += 1
            event_set_circle.append(well_arena)
    else:
        print('no events in experiments_mscore')
    
    experiment['arena circle']= event_set_circle
    experiments_mscore[fileloc_set[i]] = experiment
print('done',ii)




# imgplot = plt.imshow(frame)
# plt.show()
# print(circles.shape, circles)

05_08_24_exp1
v_05_08_24_exp1_A1 [289 291 232] (50, 3)
v_05_08_24_exp1_B1 [293 296 233] (43, 3)
v_05_08_24_exp1_D1 [286 300 232] (50, 3)
v_05_08_24_exp1_A2 [293 294 233] (33, 3)
v_05_08_24_exp1_B2 [286 302 231] (50, 3)
05_08_24_exp2
v_05_08_24_exp2_C1 [299 295 234] (50, 3)
v_05_08_24_exp2_D1 [274 298 233] (50, 3)
v_05_08_24_exp2_A2 [294 295 234] (16, 3)
v_05_08_24_exp2_B2 [289 292 234] (17, 3)
v_05_08_24_exp2_D2 [282 318 232] (50, 3)
05_09_24_exp1
v_05_09_24_exp1_A1 [280 295 233] (48, 3)
v_05_09_24_exp1_C1 [306 292 233] (50, 3)
v_05_09_24_exp1_D1 [276 296 231] (50, 3)
v_05_09_24_exp1_A2 [290 295 233] (44, 3)
v_05_09_24_exp1_B2 [283 292 234] (20, 3)
v_05_09_24_exp1_C2 [294 291 233] (41, 3)
v_05_09_24_exp1_D2 [281 317 234] (26, 3)
05_09_24_exp2
v_05_09_24_exp2_B1 [296 293 232] (50, 3)
v_05_09_24_exp2_C1 [285 292 234] (50, 3)
v_05_09_24_exp2_D1 [274 299 234] (42, 3)
v_05_09_24_exp2_A2 [291 294 232] (46, 3)
05_13_24_exp1
v_05_13_24_exp1_A1 [285 293 234] (45, 3)
v_05_13_24_exp1_D1 [291 300 

In [14]:
print(circle)

[322 291 236]


In [4]:
np.set_printoptions(precision=1)
crop_h = []
# for i in range(num_file):
# for i in range(2):
for i in range(len(experiments_mscore)):
    experiment   = experiments_mscore[fileloc_set[i]]
    arena_circle = experiment['arena circle']
    sexes = experiment['sex']
    if "events" in experiment:
        for j in range(8):
            sex = sexes[j]
            well = videofile_set[j]
            well_arena   = arena_circle[j]

            if sex != -1:
                d_std= np.sum(np.std(well_arena, axis = 0))
                circle = np.mean(well_arena, axis = 0)
                radius = circle[2]#+d_std
                center = circle[0:2]
                corner1 = circle[0:2]-radius
                corner2 = 576-(circle[0:2]+radius)


                
                

                
                
                
                if corner1[0]>0 and corner1[1]>0 and corner2[0]>0 and corner2[1]>0:
                    crop_h.append(circle)
                    print(center, radius, circle.shape)
                else:
                    print(fileloc_set[i],well,center,'wait a minute', radius)# print(crop_h.shape)
                    
# np.save('crop_h.npy', crop_h)
# np.save('experiments_mscore.npy',experiments_mscore)

[288.9 290.8] 231.6400012207031 (3,)
[293.2 296.4] 232.6069775958394 (3,)
[286.  300.2] 232.43200103759764 (3,)
[293.1 293.9] 233.29999981504497 (3,)
[286.4 302. ] 230.76000152587892 (3,)
[299.1 294.6] 233.77400024414064 (3,)
[273.5 298. ] 232.87200012207032 (3,)
[294.  295.2] 233.8812484741211 (3,)
[288.6 292. ] 233.9176483154297 (3,)
[281.7 318.4] 231.8600003051758 (3,)
[280.4 295.1] 232.64374987284342 (3,)
[305.8 291.9] 232.89399993896484 (3,)
[275.8 295.7] 231.46399963378906 (3,)
[289.8 295. ] 232.9250006242232 (3,)
[282.8 291.9] 233.62000045776367 (3,)
[293.7 291.5] 233.10487961187596 (3,)
[281.2 316.8] 234.11923159085788 (3,)
[296.  292.7] 231.66199981689454 (3,)
[285.5 291.9] 234.4559994506836 (3,)
[273.6 299.3] 234.15952337355841 (3,)
[291.2 294.4] 231.72608682383662 (3,)
[285.2 292.9] 233.8888895670573 (3,)
[291.  299.9] 232.93799987792968 (3,)
[296.4 295.6] 232.02499989100866 (3,)
[290.  297.2] 231.8599987792969 (3,)
[279.2 319.9] 232.43200012207032 (3,)
[290.4 295. ] 232.431

In [ ]:
def detect_circles(image_path):
    """
    Detects circles in an image using the Hough Circle Transform.

    Args:
        image_path: Path to the image file.

    Returns:
        A list of detected circles, where each circle is represented by 
        a tuple of (x, y, radius).
    """

    # Load the image
    img = cv2.imread(image_path)

    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Blur the image to reduce noise
    gray_blurred = cv2.medianBlur(gray, 5)

    # Detect circles using Hough Circle Transform
    circles = cv2.HoughCircles(gray_blurred, cv2.HOUGH_GRADIENT, 1, 20, 
                              param1=50, param2=30, minRadius=0, maxRadius=0)

    # Ensure at least some circles were found
    if circles is not None:
        # Convert the (x, y) coordinates and radius of the circles to integers
        circles = np.round(circles[0, :]).astype("int")

        # Draw detected circles on the original image
        for (x, y, r) in circles:
            # Draw the outer circle
            cv2.circle(img, (x, y), r, (0, 255, 0), 2)
            # Draw the center of the circle
            cv2.circle(img, (x, y), 2, (0, 0, 255), 3)

    # Display the resulting frame
    cv2.imshow('Detected Circles', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    return circles

# # Example usage:
# image_path = "path/to/your/image.jpg"  # Replace with the actual image path
# detected_circles = detect_circles(image_path)

# if detected_circles is not None:
#     print("Detected circles:", detected_circles)
# else:
#     print("No circles detected.")

In [ ]:

first_frame = 1000
last_frame = 125000
common_difference = frame_rate*30
event = np.arange(first_frame,last_frame, common_difference)
np.set_printoptions(precision=1)


for i in range(num_file):
# for i in range(len(experiments_mscore)):
# for i in range(1,num_file):
    experiment = experiments_mscore[fileloc_set[i]]
    sexes = experiment['sex']
    gtypes = experiment['genotype']
    event_set_circle = []
    if "events" in experiment:
        events = experiment['events']
        experiment_index = experiments_index[i]
        print(experiment_index)
        for j in range(8):
#         for j in range(1):
            well_arena = np.empty((0,3))
            well_food  = np.empty((0,3))
            well = videofile_set[j]
            sex = sexes[j]
            gtype = gtypes[j]
            output_file_prefix = 'v_'+experiment_index+'_'+well
            if sex != -1:
                # clipping videos for labeling and move them into a single directory
                video   = output_fileloc + OG_recording_folder +'/R72/'+output_file_prefix+'.mp4'
                print(video)
                cap = cv2.VideoCapture(video)
                length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
                n_circles = 0
                for region in event:
                    cap.set(cv2.CAP_PROP_POS_FRAMES, region)
                    res, frame = cap.read()
                    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                    gray_blurred = cv2.medianBlur(gray, 5)
                    arena = cv2.HoughCircles(gray_blurred, cv2.HOUGH_GRADIENT, 1, 20, 
                                               param1=50, param2=50, minRadius=220, maxRadius=250)
#                         food = cv2.HoughCircles(gray_blurred, cv2.HOUGH_GRADIENT, 1, 20, 
#                                                    param1=55, param2=55, minRadius=50, maxRadius=100)
                    if arena is not None:
                        well_arena = np.vstack([well_arena, (arena[0,:])[0,:]])
                        n_circles = n_circles+1
                    if n_circles >= 20:
                        break
                cap.release()
                
                cv2.destroyAllWindows()
                circle = np.mean(well_arena, axis = 0)
                circle = np.round(circle).astype("int")

                cv2.circle(frame, (circle[0], circle[1]), 240, (0, 255, 0), 2)
                # Draw the center of the circle
                cv2.circle(frame, (circle[0], circle[1]), 2, (0, 0, 255), 3)
                image   = output_fileloc + cropped_recording_folder +'/'+output_file_prefix+'.png'
                cv2.imwrite(image, frame, [cv2.IMWRITE_PNG_COMPRESSION, 0])
            event_set_circle.append(well_arena)
    else:
        print('no events in experiments_mscore')
    
    experiment['arena circle']= event_set_circle
    experiments_mscore[fileloc_set[i]] = experiment
print('done')




# imgplot = plt.imshow(frame)
# plt.show()
# print(circles.shape, circles)

In [11]:



for i in range(num_file):
    experiment = experiments_mscore[fileloc_set[i]]
    sexes = experiment['sex']
    gtypes = experiment['genotype']
    event_set_circle = []
    if "events" in experiment:
        events = experiment['events']
        events_type  = experiment['events type']
        experiment_index = experiments_index[i]
        for j in range(8):
#         for j in range(1):
            well_arena = np.empty((0,3))
            well_food  = np.empty((0,3))
            well = videofile_set[j]
            sex = sexes[j]
            gtype = gtypes[j]
            events_type_j = events_type[j]
            output_file_prefix = 'v_'+experiment_index+'_'+well
            if sex != -1:
                if sex == 0:
                    if gtype == 2:
#                         print(output_file_prefix)
                        n_event = len(events_type_j)
                        if n_event == 0:
                            print(0)
                        else:
                            print(len(events_type_j[events_type_j==1]))
                        

    else:
        print('no events in experiments_mscore')

print('done')






14
3
40
49
76
12
37
19
10
16
27
20
31
27
1
done
